<a href="https://colab.research.google.com/github/frank-895/machine_learning_journey/blob/main/random_forests/random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from fastai.imports import *

# Random Forests on the Spaceship Titanic

Inspired by my recent work on the classic Titanic dataset, I've decided to make a submission to a Kaggle competition called the [Spaceship Titanic](https://www.kaggle.com/competitions/spaceship-titanic/data?select=test.csv). The problem is set in 2912 and a group of passengers on an interstellar passenger liner have been launched into alternate dimension. The goal is to use their statistics to determine who has been launched into the alternate dimension!

Since my current area of learning is in **random forests**, I will be limiting myself to this machine learning technique for my submission to demonstrate my learning.

## Data Extraction and Cleaning

### Extraction

We can read in our dataset using Pandas, which will be useful for cleaning the data and feature engineering too.

In [53]:
df = pd.read_csv("train.csv")
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


### Data Cleaning


The first thing to check in our dataframe is for missing values. We can check this easily with Pandas.

In [54]:
df.isnull().sum()

,0
PassengerId,0
HomePlanet,201
CryoSleep,217
Cabin,199
Destination,182
Age,179
VIP,203
RoomService,181
FoodCourt,183
ShoppingMall,208


For the rows with missing values, it would be wasteful to remove them. In order to ensure all the data can be used, I will fill `NaN` values with the mode of the column they exist in. This will ensure the remaining data can be used.

In [55]:
%%capture
for col in df.columns:
  df[col] = df[col].fillna(df[col].mode()[0])

Let's make sure this function worked!

In [56]:
df.isnull().sum()

,0
PassengerId,0
HomePlanet,0
CryoSleep,0
Cabin,0
Destination,0
Age,0
VIP,0
RoomService,0
FoodCourt,0
ShoppingMall,0


Perfect!

### Feature Engineering

Let's have a look our dataframe and decide what we will do with all our columns and what **feature engineering** we have to perform to feed the data into the random forest.

In [57]:
df.describe(include='all')

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
count,8693,8693,8693,8693,8693,8693.000000,8693,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693,8693
unique,8693,3,2,6560,3,NaN,2,NaN,NaN,NaN,NaN,NaN,8473,2
top,0001_01,Earth,False,G/734/S,TRAPPIST-1e,NaN,False,NaN,NaN,NaN,NaN,NaN,Alraium Disivering,True
freq,1,4803,5656,207,6097,NaN,8494,NaN,NaN,NaN,NaN,NaN,202,4378
mean,NaN,NaN,NaN,NaN,NaN,28.728517,NaN,220.009318,448.434027,169.572300,304.588865,298.261820,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,14.355438,NaN,660.519050,1595.790627,598.007164,1125.562559,1134.126417,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,20.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,27.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,37.000000,NaN,41.000000,61.000000,22.000000,53.000000,40.000000,NaN,NaN


Here's my action plan for each feature:

- `PassengerId` is unique for each passenger; however, it does contain useful information as it is in the form `gggg_pp` where `gggg` represents the group they are travelling with. Dealing with this value is difficult and there's likely a number of good approaches. I will try transforming this into a binary feature - *whether the passenger is travelling as part of a group or not*.
- `HomePlanet`, `CryoSleep`, `Destination`, `VIP` have unique values between 2-3 so these will be converted to categorical variables without further manipulation.
- `Cabin` is more complex and requires further inspection. It is split into the format `D/N/M` so it may be possible to split this column into 3 separate categorical variables.
- `Age`, `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck` all represent continuous variables and can be fed directly into the model without further manipulation.
- `Name` is an interesting column. For now, we will dump this column as it will require significant processing to elicit useful information, especially in a random forest. It is possible that the last name could provide valuable insights for the model; however, I theorise that this information will be automatically contained within the `gggg` part of `PassengerId`.

Then finally, `Transported` is our target variable. This represents either 'True' or 'False'.

#### PassengerId

We need to extract the first 4 digits of each value in the `PassengerId` column and convert the value to an integer. Then we can determine if this integer is unique in the column or not, which will be used to determine if the passenger is travelling as part of a group or not.

In [58]:
def process_passenger_id(df):
  # first 4 digits
  df['PassengerId'] = [int(i[:4]) for i in df['PassengerId']]
  # determine uniqueness
  df['with_group'] = df['PassengerId'].map(df['PassengerId'].value_counts() > 1).astype(int)

process_passenger_id(df)
df['PassengerId']

#### Cabin

In [74]:
df['Cabin'].head()

,Cabin
0,B/0/P
1,F/0/S
2,A/0/S
3,A/0/S
4,F/1/S


For each value in `Cabin` there are three separate pieces of information `deck/num/side`.

Thus, we will split `Cabin` into 3 separate features. `deck` and `side` have a limited number of values and will be **categorical variables**. `num` is a continuous variable and may provide useful information about a person's chance of being in an alternate dimension if it depends on their location in the ship.

In [78]:
df[['deck', 'num', 'side']] = df['Cabin'].str.split('/', expand=True)
df[['deck', 'num', 'side']]


,deck,num,side
0,B,0,P
1,F,0,S
2,A,0,S
3,A,0,S
4,F,1,S
...,...,...,...
8688,A,98,P
8689,G,1499,S
8690,G,1500,S
8691,E,608,S


Let's make sure that there are no null values - in case the format was not consistent for the data.

In [81]:
df[['deck', 'num', 'side']].isnull().sum()

,0
deck,0
num,0
side,0


Perfect, now let's confirm that the theorised assignment of categorical and continuous variables was suitable.

In [79]:
df[['deck', 'num', 'side']].nunique()

,0
deck,8
num,1817
side,2


Seems good! `num` has many unique values, so this will likely be better as a continuous variable.

In [82]:
df['num'] = df['num'].astype(int)

Now I just need to finish preparing all continuous and categorical variables, since all features have been created.

#### Categorical Variables

When creating random forests, we don't need to create dummy variables for non-numeric columns, instead just convert them to **categorical variables** with Pandas. Internally, they will be interpreted as numbers.

In [83]:
categories = ['HomePlanet', 'Destination', 'CryoSleep', 'VIP', 'deck', 'side']
def categorical(df):
  for c in categories:
    df[c] = pd.Categorical(df[c])

categorical(df)
df[categories]

,HomePlanet,Destination,CryoSleep,VIP,deck,side
0,Europa,TRAPPIST-1e,False,False,B,P
1,Earth,TRAPPIST-1e,False,False,F,S
2,Europa,TRAPPIST-1e,False,True,A,S
3,Europa,TRAPPIST-1e,False,False,A,S
4,Earth,TRAPPIST-1e,False,False,F,S
...,...,...,...,...,...,...
8688,Europa,55 Cancri e,False,True,A,P
8689,Earth,PSO J318.5-22,True,False,G,S
8690,Earth,TRAPPIST-1e,False,False,G,S
8691,Europa,55 Cancri e,False,False,E,S


#### Continuous Variables

The continuous variables should be all in numeric format, but we will confirm that and we should be ready!

In [90]:
cont = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'num']
df[cont].dtypes

,0
Age,float64
RoomService,float64
FoodCourt,float64
ShoppingMall,float64
Spa,float64
VRDeck,float64
num,int64


#### Finalising

Now, I want to put all of the feature engineering into a single function - this will make it easy to perform on the test set. I will also drop columns that we will not be using.

In [91]:
def preprocessing(df):
  pass